# Natural Language Processing

P.S. to run again the code, please use the .csv files existed in this .zip.
The data columns had to be named. so, a slight change in .txt files load data into pandas dataframes.

## Importing the libraries

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
from sklearn.metrics import confusion_matrix, accuracy_score

# Importing the dataset

In [2]:
dataset = pd.read_csv('r8-train-all-terms.txt', delimiter = '\t', quoting = 3)
test_dataset = pd.read_csv('r8-test-all-terms.txt', delimiter = '\t', quoting = 3)
dataset = dataset[pd.notnull(dataset['Example'])]
test_dataset = test_dataset[pd.notnull(dataset['Example'])]

test_dataset

C:\Users\user\AppData\Local\Temp/ipykernel_10136/653030402.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_dataset = test_dataset[pd.notnull(dataset['Example'])]


,Class,Example
0,trade,asian exporters fear damage from u s japan rif...
1,grain,china daily says vermin eat pct grain stocks a...
2,ship,australian foreign ship ban ends but nsw ports...
3,acq,sumitomo bank aims at quick recovery from merg...
4,earn,amatil proposes two for five bonus share issue...
...,...,...
2184,money-fx,balladur insists on maintenance of louvre acco...
2185,trade,philippine trade gap widens in january august ...
2186,crude,iran soviet union to swap crude refined produc...
2187,acq,n z s chase corp makes offer for entregrowth c...


# Cleaning the texts

In [3]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
replace_by_space_re = re.compile('[/(){}\[\]\|@,;]')
bad_symbols_re = re.compile('[^0-9a-z #+_]')

In [5]:
corpus = []
for i in range(0,5485):
    example = re.sub ('[^a-zA-Z]', ' ', dataset['Example'][i])
    example = example.lower()
    example = example.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    example = [ps.stem(word) for word in example if not word in set(all_stopwords)]
    example = ' '.join(example)
    corpus.append(example)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

test_corpus= []
for i in range(0, 2189):
    example = re.sub('[^a-zA-Z]', ' ', test_dataset['Example'][i])
    example = example.lower()
    example = example.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    # all_stopwords.remove('not')
    example = [ps.stem(word) for word in example if not word in set(all_stopwords)]
    example = ' '.join(example)
    test_corpus.append(example)

# Splitting Datasets:

In [7]:
corpus = [str(item) for item in corpus]

X_train = corpus
y_train = dataset['Class']

X_test = test_corpus
y_test = test_dataset['Class']

# Naive Bayes Classifier

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

In [9]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [10]:
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

In [11]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 662    0   34    0    0    0    0    0]
 [  32   69   20    0    0    0    0    0]
 [  10    0 1073    0    0    0    0    0]
 [   5    0    5    0    0    0    0    0]
 [  32    0   37    0   10    2    0    0]
 [  59    0   15    0    0   13    0    0]
 [  28    3    3    0    0    0    0    2]
 [  20    0   19    0    0    0    0   36]]


0.8510735495660119

In [12]:
# Tokenizing
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5485)

# Linear Support Vector Machine:

In [13]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         acq       0.93      0.99      0.96       696
       crude       0.98      0.94      0.96       121
        earn       0.99      0.99      0.99      1083
       grain       1.00      0.80      0.89        10
    interest       0.92      0.85      0.88        81
    money-fx       0.93      0.75      0.83        87
        ship       1.00      0.72      0.84        36
       trade       0.91      0.96      0.94        75

    accuracy                           0.97      2189
   macro avg       0.96      0.88      0.91      2189
weighted avg       0.97      0.97      0.96      2189



In [14]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 689    0    6    0    0    0    0    1]
 [   7  114    0    0    0    0    0    0]
 [  13    0 1070    0    0    0    0    0]
 [   1    0    0    8    1    0    0    0]
 [   3    0    1    0   69    5    0    3]
 [  14    0    1    0    5   65    0    2]
 [   7    2    0    0    0    0   26    1]
 [   3    0    0    0    0    0    0   72]]


0.9652809502055734

# Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])

logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print(classification_report(y_test, y_pred))

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

         acq       0.98      0.98      0.98       696
       crude       0.96      0.97      0.96       121
        earn       0.99      0.99      0.99      1083
       grain       1.00      0.90      0.95        10
    interest       0.93      0.83      0.88        81
    money-fx       0.87      0.93      0.90        87
        ship       0.92      0.92      0.92        36
       trade       0.94      0.99      0.96        75

    accuracy                           0.98      2189
   macro avg       0.95      0.94      0.94      2189
weighted avg       0.98      0.98      0.98      2189

